In [1]:
import tensorflow as tf
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Flatten,Dropout, LSTM, BatchNormalization,SimpleRNN,Input,Conv1D,Conv2D,Concatenate,Activation,MaxPooling1D,Masking,GRU,Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from tensorflow.keras import backend as K
from tensorflow.keras import layers
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score

import sklearn.metrics as metrics

import tensorflow_addons as tfa
from tensorflow.keras.callbacks import LambdaCallback

from sklearn import preprocessing
import keras

import csv

from keras.callbacks import Callback, ModelCheckpoint
from keras.models import load_model
from keras.wrappers.scikit_learn import KerasClassifier

from keras.callbacks import CSVLogger

In [251]:
df = pd.read_csv(r"C:\Users\brian\Desktop\Python\Mayo Model\Data_outcome_deidentified.csv", header=0)
df.drop(['POST','PRE','causeofdeath','primcauseofdeath','death.info','numdisvessels','maxlesionlength', 'wposttimi', 'numbms','numdes', 'numstents', 'maxdevdiameter', 'maxdevlength','otherventsupport\nYes:1 \nNo:0'],axis=1,inplace=True)
effect = ['Inhospital mortality\nYes:1\nNo:0','Mortality 6 months\nYes:1\nNo:0', 'Mortality 1 yr\nYes:1\nNo:0','Mortality 2 yr\nYes:1\nNo:0', 'Mortality 5yr\nYes:1\nNo:0','Time from PCI to Stroke_6mo', 'Time from PCI to Stroke_1yr','Time from PCI to Stroke_2yr', 'Time from PCI to Stroke_5yr']
causes =  df.drop(effect,axis=1)

C:\Users\brian\AppData\Local\Temp\ipykernel_70056\3896751810.py:1: DtypeWarning: Columns (172) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\brian\Desktop\Python\Mayo Model\Data_outcome_deidentified.csv", header=0)


In [296]:
test_conf = pd.read_csv(r"C:\Users\brian\Desktop\Python\Mayo Model\test_confidence.csv", header=0)
test_conf = test_conf.drop(['Unnamed: 0'], axis=1)
train_conf = pd.read_csv(r"C:\Users\brian\Desktop\Python\Mayo Model\train_confidence.csv", header=0)
train_conf = train_conf.drop(['Unnamed: 0'], axis=1)

f1_cutoff = [0.618908763,0.874658227,0.84739238,0.341690063,0.216099053621292,0.228994444,0.009523808,0,0]
cf_cutoff = [0.4,0.413742632,0.257319182,0.271106124,0.254969954,0,0,0,0]

train_best = [0.54, 0.4, 0.28, 0.0, 0.24, 0, 0, 0, 0]
test_best = [0.02, 0.1, 0.3, 0.16, 0.14, 0, 0, 0, 0]

test_ranges = [6365, 5931, 5211, 4348, 3179, 5599, 5072, 4154, 2104]
train_ranges = [6364, 5930, 5210, 4347, 3179, 5598, 5071, 4153, 2104]

In [32]:
test_cutoffs = []
train_cutoffs = []

In [140]:
def recall(tp, tn, fp, fn):
    if (tp + fn) == 0:
        return 0
    return tp / (tp + fn)

def precision(tp, tn, fp, fn):
    if (tp + fp) == 0:
        return 0
    return tp / (tp + fp)

def f1(precision, recall):
    if (precision + recall) == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

In [86]:
def singleCutoff(data, range, cutoff, eff):
    results = []
    data = data.iloc[:range:]

    for j in data[eff]:
        if cf_cutoff[count] == 0:
            results.append(0)
        elif j > cutoff:
            results.append(1)
        else:
            results.append(0)

    return results

In [127]:
def singleCompare(data, array):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    counter = 0
    for x in array:
        if x == 1 and data[counter] == 1:
            tp += 1
        elif x == 1 and data[counter] == 0:
            fn += 1
        elif x == 0 and data[counter] == 1:
            fp += 1
        else:
            tn += 1
        counter += 1
    
    return tp, tn, fp, fn


In [159]:
best_train_cf = []
best_test_cf = []
count = 0
for e in effect:
    #print(e, "\n")
    train_data = []
    test_data = []
    best_train_score = 0
    best_train_cut = 0
    best_test_score = 0
    best_test_cut = 0
    for i in range(51):
        cutoff = i/50
        train_data = singleCutoff(train_conf, train_ranges[count], cutoff, e)
        test_data = singleCutoff(test_conf, test_ranges[count], cutoff, e)
        train_tp, train_tn, train_fp, train_fn = singleCompare(train_data, y_trains[count])
        test_tp, test_tn, train_fp, train_fn = singleCompare(test_data, y_tests[count])
        train_precision = precision(train_tp, train_tn, train_fp, train_fn)
        train_recall = recall(train_tp, train_tn, train_fp, train_fn)
        test_precision = precision(test_tp, test_tn, train_fp, train_fn)
        test_recall = recall(test_tp, test_tn, train_fp, train_fn)
        train_f1 = f1(train_precision, train_recall)
        test_f1 = f1(test_precision, test_recall)
        

        if train_f1 > best_train_score:
            best_train_score = train_f1
            best_train_cut = cutoff

        if test_f1 > best_test_score:
            best_test_score = test_f1
            best_test_cut = cutoff

    best_train_cf.append(best_train_cut)
    best_test_cf.append(best_test_cut)
        
    count += 1
        
    

In [145]:
print(best_train_cf)
print(best_test_cf)


[0.54, 0.4, 0.28, 0.0, 0.24, 0, 0, 0, 0]
[0.02, 0.1, 0.3, 0.16, 0.14, 0, 0, 0, 0]


In [230]:
def getCutoffs(test_data,ranges,cut_scores):
    saved_data = test_data
    test_results = []
    count = 0

    for e in effect:
        test_results.append([])

        test_data = saved_data.iloc[:ranges[count]:]

        for j in test_data[e]:
            if cut_scores[count] == 0:
                test_results[count].append(0)
            elif j > cut_scores[count]:
                test_results[count].append(1)
            else:
                test_results[count].append(0)
        count += 1

    return test_results

In [232]:
test_cutoffs = getCutoffs(test_conf,test_ranges,test_best)
train_cutoffs = getCutoffs(train_conf,train_ranges,train_best)

In [371]:
y_trains = []
y_tests = []
count = 0
for e in effect:
    X= df[df[e]>-1]
    
    if(e =='Mortality 1 yr\nYes:1\nNo:0'):
        X=X[X['Mortality 6 months\nYes:1\nNo:0']!=1]
    elif(e =='Mortality 2 yr\nYes:1\nNo:0' ):
        X=X[(X['Mortality 6 months\nYes:1\nNo:0']!=1) & (X['Mortality 1 yr\nYes:1\nNo:0']!=1)]
    elif(e =='Mortality 5yr\nYes:1\nNo:0'):
        X = X[(X['Mortality 6 months\nYes:1\nNo:0']!=1) & (X['Mortality 1 yr\nYes:1\nNo:0']!=1) & (X['Mortality 2 yr\nYes:1\nNo:0']!=1) ]
    if(e =='Time from PCI to Stroke_1yr'):
        X=X[X['Time from PCI to Stroke_6mo']!=1]
    elif(e =='Time from PCI to Stroke_2yr' ):
        X=X[(X['Time from PCI to Stroke_6mo']!=1) & (X['Time from PCI to Stroke_1yr']!=1)]
    elif(e =='Time from PCI to Stroke_5yr'):
        X = X[(X['Time from PCI to Stroke_6mo']!=1) & (X['Time from PCI to Stroke_1yr']!=1) & (X['Time from PCI to Stroke_2yr']!=1) ]


    y= X[e]
    X=X.drop(effect,axis=1)
    #print(y.iloc[:100])
    #X_train,X_test = X.iloc[:len(X)//2,:],X.iloc[len(X)//2:,:]
    y_train,y_test = y.iloc[:len(y)//2-1],y.iloc[len(y)//2+1:]
    y_trains.append(y_train)
    y_tests.append(y_test)
    print(len(y_train))


6364
5930
5210
4347
2543
5598
5071
4153
2104


In [234]:
def compareResults(newData, y_array):
    result = []
    for i in range(9):
        result.append([])
        count = 0
        for x in y_array[i]:
            if x == 1 and newData[i][count] == 1:
                result[i].append('tp')
            elif x == 1 and newData[i][count] == 0:
                result[i].append('fn')
            elif x == 0 and newData[i][count] == 1:
                result[i].append('fp')
            else:
                result[i].append('tn')
            count += 1
    return result


In [247]:
test_results = compareResults(test_cutoffs, y_tests)
train_results = compareResults(train_cutoffs, y_trains)

In [301]:
def probability(data, ranges):
    saved_data = causes
    count = 0
    true_positives = []
    true_negatives = []
    false_positives = []
    false_negatives = []
    for e in effect:
        true_positives.append([])
        true_negatives.append([])
        false_positives.append([])
        false_negatives.append([])
        for i in causes:
            tp_count = 0
            tn_count = 0
            fp_count = 0
            fn_count = 0
            total = 0

            comp_data = saved_data.iloc[:ranges[count]:]

            #print(ranges[count])
            #print(len(comp_data))
            #print(len(data[count]))

            index = 0
            for x in comp_data[i]:
                if x == 1:
                    if data[count][index] == 'tp':
                        tp_count += 1
                    if data[count][index] == 'tn':
                        tn_count += 1
                    if data[count][index] == 'fp':
                        fp_count += 1
                    if data[count][index] == 'fn':
                        fn_count += 1 
                    total += 1
                index += 1
            
            if total > 0:
                tps = tp_count / total
                tns = tn_count / total
                fps = fp_count / total
                fns = fn_count / total

            #print(tps, tns, fps, fns)

            true_positives[count].append(tps)
            true_negatives[count].append(tns)
            false_positives[count].append(fps)
            false_negatives[count].append(fns)

        count += 1

    return true_positives, true_negatives, false_positives, false_negatives


In [312]:
test_tp, test_tn, test_fp, test_fn = probability(test_results, test_ranges)
train_tp, train_tn, train_fp, train_fn = probability(train_results, train_ranges)


In [313]:
cause = []
for c in causes:
    cause.append(c)

fn1


In [323]:
train_prob = pd.DataFrame()
test_prob = pd.DataFrame()
train_prob['names'] = cause
test_prob['names'] = cause

In [365]:
tpn = 'tp'
tnn = 'tn'
fpn = 'fp'
fnn = 'fn'

In [366]:
cnt = 8
tpn += '8'
tnn += '8'
fpn += '8'
fnn += '8'
print(tpn)

tp8


In [367]:
train_prob[tpn] = train_tp[cnt]
train_prob[tnn] = train_tn[cnt]
train_prob[fpn] = train_fp[cnt]
train_prob[fnn] = train_fn[cnt]

test_prob[tpn] = test_tp[cnt]
test_prob[tnn] = test_tn[cnt]
test_prob[fpn] = test_fp[cnt]
test_prob[fnn] = test_fn[cnt]

In [370]:
train_prob.to_csv(r"C:\Users\brian\Desktop\Python\Mayo Model\train_prob.csv")
test_prob.to_csv(r"C:\Users\brian\Desktop\Python\Mayo Model\test_prob.csv")

In [239]:
def normalizeArray(input, limit):
    for i in range(9):
        while len(input[i]) < limit:
            input[i].append(0)
    return input


In [240]:
def saveAsCSV(input, path):
    new = pd.Dataframe()
    count = 0
    for e in effect:
        new[e] = input[count]
        count += 1
    new.to_csv(path)

In [241]:
test_results = normalizeArray(test_results, 6365)
train_results = normalizeArray(train_results, 6364)
test_cutoffs = normalizeArray(test_cutoffs, 6365)
train_cutoffs = normalizeArray(train_cutoffs, 6364)
#saveAsCSV(test_results, r"C:\Users\brian\Desktop\Python\Mayo Model\test_result.csv")
#saveAsCSV(train_results, r"C:\Users\brian\Desktop\Python\Mayo Model\train_result.csv")

In [246]:
temp = pd.DataFrame()
count = 0
for e in effect:
    temp[e] = test_cutoffs[count]
    count += 1
temp.to_csv(r"C:\Users\brian\Desktop\Python\Mayo Model\test_cutoffs.csv")